# Moon Analysis 2024
## JCH - Nov. 2024

This notebook attemps to integrate the Moon analysis from previous Notebooks (`Analysis Moon 2024` and `Moon-FWHM-Offsets-2024`) into a single runable function in order to be able to update the calibration files with the measured focal plane rotation from `Moon-FWHM-Offsets-2024`. 

- `Analysis Moon 2024` is doing a minimal TOD treatment to be able to perform coadded maps of the Moon for each TES where one can see the Moon main peak, sometimes secondary peaks as well as the trees in fron t of the Salta Intergation hall door. This notebook produces files with Moon maps for each TES.
- `Moon-FWHM-Offsets-2024` is reading those Moon maps for each TES and focuses on the main peak from the Moon. It then fits its location, amplitude and FWHM with a 2D Gaussian. Then, comparing the location of the main peak with those predicted from Maynooth's optical modeling, one calculates the average translation and rotation of the focal plane with respect to ideality. It finally shows a focal plane map of the Moon images that shows the Moon at the center of the image for all TES. In the cases where it is not right at the center, then we need to understand the reasons that could include wring labeling of some TES (w.r.t. wiring scheme) or undetected Moon.

So the idea here is to be able to: 
- perform the analysis first with the ideal locations of the focal plane
- calculate the updated corresponding calibration file
- redo the analysis with the updated calibration file
- check that the new translation / rotation is compatible with identity
- the non satisfying TES will need to be studied in details.


## Imports and notebook configuration

In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))



### General imports
import os
import sys
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from datetime import datetime
from joblib import Parallel, delayed
from multiprocessing import Manager, Lock
from scipy.signal import medfilt
from scipy.interpolate import interp1d
import healpy as hp


plt.rc('figure',figsize=(20,12))
plt.rc('font',size=12)

### Astropy configuration
from astropy.visualization import astropy_mpl_style, quantity_support
quantity_support()
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_moon, get_sun



#### QUBIC IMPORT
from qubicpack.utilities import Qubic_DataDir
from qubicpack.qubicfp import qubicfp
from qubicpack.pix2tes import pix2tes, tes2pix

from qubic.lib.Calibration import Qfiber
from qubic.lib.Calibration import Qselfcal
from qubic.lib.Qutilities import find_file, progress_bar
from qubic.lib.QdataHandling import display_healpix_map, identify_scans
from qubic.lib import Qdictionary 
from qubic.lib.Instrument import Qinstrument


def iQS2iQP(indexQS):
    qpnumi, qpasici = qp.pix2tes.pix2tes(indexQS+1)
    return qpnumi+(qpasici-1)*128-1

def iQP2iQS(indexQP):
    QStesnum = qp.pix2tes.tes2pix(indexQP%128+1, indexQP//128+1)
    return QStesnum-1

d = Qdictionary.qubicDict()
dictfilename = '/dicts/global_source_oneDet.dict'
d.read_from_file(dictfilename)
q = Qinstrument.QubicInstrument(d)


### Temporary update of the path 
in order to be able to load libraries that are still in development and not yet in the QUBIC path. This will have to be removed when the relevant libraries are finalized and integrated into QubicSoft

In [ ]:
dirtemplibs = [os.environ['QUBIC_DATADIR']+'scripts/DiversJC/CalibSalta/']
for rep in dirtemplibs:     
    if rep not in sys.path:
        sys.path.append(rep)

#### Local files that will need to be installed in the Qubic Libs
import fitting as fit
import time_domain_tools as tdt

## A number of functions that will be used throughout this notebook
They should be ultimately well commented and integrated into actual libraries in QubicSoft of course.

In [ ]:
def healpix_map(azt, elt, tod, flags=None, flaglimit=0, nside=128, countcut=0, unseen_val=hp.UNSEEN):
    if flags is None:
        flags = np.zeros(len(azt))
    
    ok = flags <= flaglimit 
    return healpix_map_(azt[ok], elt[ok], tod[ok], nside=nside, countcut=countcut, unseen_val=unseen_val)


def healpix_map_(azt, elt, tod, nside=128, countcut=0, unseen_val=hp.UNSEEN):
    ips = hp.ang2pix(nside, azt, elt, lonlat=True)
    mymap = np.zeros(12*nside**2)
    mapcount = np.zeros(12*nside**2)
    for i in range(len(azt)):
        mymap[ips[i]] += tod[i]
        mapcount[ips[i]] += 1
    unseen = mapcount <= countcut
    mymap[unseen] = unseen_val
    mapcount[unseen] = unseen_val
    mymap[~unseen] = mymap[~unseen] / mapcount[~unseen]
    return mymap, mapcount

def display_one(mapsb, anatype='', sub=(1,1,1), nlo=3, nhi=3, reso=12, rot=[0,50]):
    unseen = (mapsb == hp.UNSEEN)
    mm, ss = Qfiber.meancut(mapsb[~unseen], 3)
    hp.gnomview(mapsb, rot=rot, reso=reso, sub=sub, title=anatype+'\n Both scans $\sigma$ = {0:5.3g}'.format(ss), min=-nlo*ss, max=nhi*ss)


def do_display_all(mapsb, mapsb_pos, mapsb_neg, mapav, mapdiff, mapdiff2, rot=[0,50], anatype='', reso=12, myrange=None, TESNum = None):
    unseen = (mapsb == hp.UNSEEN) | (mapsb_pos == hp.UNSEEN) | (mapsb_neg == hp.UNSEEN)
    mm, ss = Qfiber.meancut(mapsb[~unseen], 3)
    
    if myrange is None:
        mini = -3*ss
        maxi = 3*ss
    else:
        mini = myrange[0]
        maxi = myrange[1]
        
    if TESNum != None:
        anatype += '\n TES# {}'.format(TESNum)

    plt.figure()
    hp.gnomview(mapsb, rot=rot, reso=reso, sub=(2,3,1), title=anatype+'\n Both scans $\sigma$ = {0:5.4g}'.format(ss), min=mini, max=maxi)
    mmp, ssp = Qfiber.meancut(mapsb_pos[~unseen], 3)
    hp.gnomview(mapsb_pos, rot=rot, reso=reso, sub=(2,3,2), title=anatype+'\n Pos scans $\sigma$ = {0:5.4g}'.format(ssp), min=mini, max=maxi)
    mmn, ssn = Qfiber.meancut(mapsb_neg[~unseen], 3)
    hp.gnomview(mapsb_neg, rot=rot, reso=reso, sub=(2,3,3), title=anatype+'\n Neg scans $\sigma$ = {0:5.4g}'.format(ssn), min=mini, max=maxi)
    mma, ssa = Qfiber.meancut(mapav[~unseen], 3)
    hp.gnomview(mapav, rot=rot, reso=reso, sub=(2,3,4), title=anatype+'\n Av of Both scans $\sigma$ = {0:5.4g}'.format(ssa), min=mini, max=maxi)
    mmd, ssd = Qfiber.meancut(mapdiff[~unseen], 3)
    hp.gnomview(mapdiff, rot=rot, reso=reso, sub=(2,3,5), title=anatype+'\n Diff of both scans $\sigma$ = {0:5.4g}'.format(ssd), min=mini/ss*ssd, max=maxi/ss*ssd)
    mmd2, ssd2 = Qfiber.meancut(mapdiff2[~unseen], 3)
    hp.gnomview(mapdiff2, rot=rot, reso=reso, sub=(2,3,6), title=anatype+'\n Both - Av $\sigma$ = {0:5.4g}'.format(ssd2), min=mini/ss**ssd, max=maxi/ss*ssd)
    

def display_all(mapsb, mapsb_pos, mapsb_neg, anatype='', rot=[0,50], highcontrast=False, reso=12, myrange=None, TESNum=None):
    unseen = (mapsb == hp.UNSEEN) | (mapsb_pos == hp.UNSEEN) | (mapsb_neg == hp.UNSEEN)

    ### Average of back and Forth
    mapav = (mapsb_pos + mapsb_neg)/2
    mapav[unseen] = hp.UNSEEN

    ### Difference of back and Forth
    mapdiff = (mapsb_pos - mapsb_neg)
    mapdiff[unseen] = hp.UNSEEN

    ### Difference of All and Av
    mapdiff2 = (mapav - mapsb)
    mapdiff2[unseen] = hp.UNSEEN
    
    if highcontrast:
        myrange = [-np.max(mapsb[~unseen])/10, np.max(mapsb[~unseen])*0.8]
        
    do_display_all(mapsb, mapsb_pos, mapsb_neg, mapav, mapdiff, mapdiff2, rot=rot, anatype=anatype, reso=reso, myrange=myrange, TESNum=TESNum)
    
def remove_offset_scan(mytod, scantype, method='meancut', apply_to_bad = True):
    ### We remove offsets for each good scan but we also need to remove a coomparable offset for the scantype==0 reggiions in order to keep coninuity 
    ### This si donee by apply_to_bad=True
    
    indices = np.arange(len(mytod))
    last_index = 0
    myoffsetn = 0
    myoffsetp = 0
    donefirst = 0
    
    nscans = np.max(np.abs(scantype))
    for n in range(1, nscans+1):
        # scan +
        ok = scantype == n
        if method == 'meancut':
            myoffsetp, _ = Qfiber.meancut(mytod[ok], 3)
        elif method == 'median':
            myoffsetp = np.median(mytod[ok])
        elif method == 'mode':
            myoffsetp = tdt.get_mode(mytod[ok])
        else:
            break
        mytod[ok] -= myoffsetp        
        if apply_to_bad:
            first_index = np.min(indices[ok])
            if (n==1) & (donefirst==0): myoffsetn = myoffsetp ### deal with first region
            vals_offsets = myoffsetn + np.linspace(0,1, first_index-last_index-1)*(myoffsetp-myoffsetn)
            mytod[last_index+1:first_index] -= vals_offsets
            last_index = np.max(indices[ok])
            donefirst = 1
        
        
        # scan -
        ok = scantype == (-n)
        if method == 'meancut':
            myoffsetn, _ = Qfiber.meancut(mytod[ok], 3)
        elif method == 'median':
            myoffsetn = np.median(mytod[ok])
        elif method == 'mode':
            myoffsetn = tdt.get_mode(mytod[ok])
        else:
            break
        mytod[ok] -= myoffsetn
        if apply_to_bad:
            first_index = np.min(indices[ok])
            if (n==1) & (donefirst==0): myoffsetp = myoffsetn ### deal with first region
            vals_offsets = myoffsetp + np.linspace(0,1, first_index-last_index-1)*(myoffsetn-myoffsetp)
            mytod[last_index+1:first_index] -= vals_offsets
            last_index = np.max(indices[ok])
            donefirst = 1
    
    return mytod


def decorel_azimuth(mytod, azt, scantype, degree=2, doplot=True):
    ### Profiling in Azimuth
    okall = np.abs(scantype) > 0 
    okpos = scantype > 0 
    okneg = scantype < 0 
    oks = [okpos, okneg]
    oks_names = ['+ scans', '- scans']
    polys = []
    if doplot:
        figure()
    for i in range(len(oks)):
        ok = oks[i]
        minaz = np.min(azt[ok])
        maxaz = np.max(azt[ok])
        xc, yc, dx, dy, _ = Qfiber.profile(azt[ok], mytod[ok], rng=[minaz, maxaz], nbins=25, mode=True, dispersion=True, plot=False)
        z = polyfit(xc, yc, degree, w=1./dy)
        p = np.poly1d(z)
        polys.append(p)
        xaz = np.linspace(minaz, maxaz, 100)
        if doplot:
            pl = errorbar(xc, yc, yerr=dy, xerr=dx, fmt='o')
            plot(xaz, p(xaz), label=oks_names[i], color=pl[0].get_color())
    if doplot:
        xlabel('Azimuth [deg]')
        ylabel('Mode of TOD')
        legend()

    ### Removing the azimuthal effect
    ok = scantype >= 0
    mytod[ok] -= polys[0](azt[ok])
    ok = scantype < 0
    mytod[ok] -= polys[1](azt[ok])
    
    return mytod

def get_chunks(mytod, scantype, value):
    ### returns chunks corresponding to a given value
    current_chunk = []
    chunk_idx = []
    inchunk = 0
    chunknum = 0
    for i in range(len(scantype)):
        if scantype[i]==value:
            inchunk = 1
            current_chunk.append(i)
        else:
            if inchunk == 1:
                chunknum += 1
                chunk_idx.append([current_chunk[0], current_chunk[len(current_chunk)-1]])
                current_chunk = []
                inchunk = 0
    if inchunk == 1:
        chunk_idx.append([current_chunk[0], current_chunk[len(current_chunk)-1]])
    return chunk_idx


def linear_rescale_chunks(mytod, chunks, sz=1000):
    for i in range(len(chunks)):
        thechunk = chunks[i]
        chunklen = thechunk[1] - thechunk[0]+1
        if thechunk[0] == 0:
            # this is the starting index => just the average
            vals = np.zeros(chunklen) + np.median(mytod[thechunk[1]+1: thechunk[1]+sz]) + np.median(mytod[thechunk[0]:thechunk[1]])
            mytod[thechunk[0]:thechunk[1]+1] -= vals
        elif thechunk[1]==(len(mytod)-1):
            # this is the last one => just the average
            vals = np.zeros(chunklen) + np.median(mytod[thechunk[0]-1-sz: thechunk[0]-1]) + np.median(mytod[thechunk[0]:thechunk[1]])
            mytod[thechunk[0]:thechunk[1]+1] -= vals
        else:
            left = np.median(mytod[thechunk[0]-1-sz: thechunk[0]-1])
            right = np.median(mytod[thechunk[1]+1: thechunk[1]+sz])
            vals = left + np.linspace(0,1, chunklen)*(right-left)
            mytod[thechunk[0]:thechunk[1]+1] -= np.median(mytod[thechunk[0]:thechunk[1]+1]) - vals
            
    return mytod

def decorel_azel(mytod, azt, elt, scantype, doplot=True, nbins=50, n_el=20, degree=None, nbspl=10):
    ### Profiling in Azimuth and elevation
    el_lims = np.linspace(np.min(elt)-0.0001, np.max(elt)+0.0001, n_el+1)
    el_av = 0.5 * (el_lims[1:] + el_lims[:-1])

    okall = np.abs(scantype) > 0 
    okpos = scantype > 0 
    okneg = scantype < 0 
    oks = [okpos, okneg]
    oks_names = ['+ scans', '- scans']
    minaz = np.min(azt[okall])
    maxaz = np.max(azt[okall])
    xaz = np.linspace(minaz, maxaz, 100)
    
    ### Use polynomials or spline fitting to remove drifts and large features
    if degree != None:
        coefficients = np.zeros((2, n_el, degree+1))
    else:
        coefficients = np.zeros((2, n_el, nbspl))

    if doplot: 
        plt.figure()
    for i in range(len(oks)):
        if doplot: 
            plt.subplot(1,2,i+1)
            plt.xlabel('Az')
            plt.ylabel('TOD')
            plt.title(oks_names[i])
        for j in range(n_el):
            ok = oks[i] & (elt >= el_lims[j]) & (elt < el_lims[j+1])
            if np.sum(ok)==0:
                break
            xc, yc, dx, dy, _ = Qfiber.profile(azt[ok], mytod[ok], rng=[minaz, maxaz], nbins=nbins, mode=True, dispersion=True, plot=False)

            if degree != None:
                ### Polynomial Fitting
                z = polyfit(xc, yc, degree, w=1./dy)
                coefficients[i,j,:] = z
                p = np.poly1d(z)
                fitted = p(xaz)
            else:
                ### Spline Fitting
                splfit = tdt.MySplineFitting(xc, yc, dy, nbspl)
                coefficients[i,j,:] = splfit.alpha
                fitted = splfit(xaz)
            if doplot:
                pl = plt.errorbar(xc, yc, yerr=dy, xerr=dx, fmt='o')
                plt.plot(xaz, fitted, color=pl[0].get_color(), label = oks_names[i] + ' - El = {0:5.1f}'.format(np.mean(elt[ok])))
    #if doplot: legend()

    ### Now interpolate this to remove it to the data
    nscans = np.max(np.abs(scantype))
    for i in range(1, nscans+1):
        okp = scantype == i
        okn = scantype == (-i)
        for ok in [okp, okn]:
            the_el = np.median(elt[ok])
            if degree != None:
                myp = np.poly1d([np.interp(the_el, el_av, coefficients[0,:,i]) for i in np.arange(degree+1)])
                mytod[ok] -= myp(azt[ok])
            else:
                myalpha = [np.interp(the_el, el_av, coefficients[0,:,i]) for i in np.arange(nbspl)]
                mytod[ok] -= splfit.with_alpha(azt[ok], myalpha)
                    
                    
    ### And interpolate for scantype==0 regions
    bad_chunks = get_chunks(mytod, scantype, 0)
    mytod = linear_rescale_chunks(mytod, bad_chunks, sz=100)
    return mytod

## Dataset and Observing site
in the case of the July 2022 Moon observations, they were from Salta CNEA Regional and the UTC offset was -3 hours.

### Location of the raw TOD files
Beware this will have to be changed for each one's configuration

In [ ]:
mydatadir = '/Users/hamilton/Qubic/Data/CommissioningTD/'

### Observation date and corresponding file

In [ ]:
ObsDate = '2022-07-14'
ObsSession = 0
dirs = glob.glob(mydatadir + '/' + ObsDate + '/*')
print(dirs)
datadir = dirs[0]


### Observing Site

In [ ]:
Salta_CNEA = {'lat':-24.731358*u.deg,
              'lon':-65.409535*u.deg,
              'height':1152*u.m,
              'UTC_Offset':-3*u.hour}
Obs_Site = Salta_CNEA

## Now the big loop on the TESs to make coadded images of the Moon data.
Here we make coadded maps in a "local" coordinate system that follows the Azimuth and Elevation of the Moon.

In [ ]:
def read_data(datadir, remove_t0=True):
    """
    Reads QUBIC raw data: time and TOD, as well azimuth, elevation and 
    their corresponding time

    Parameters
    ----------
    datadir : string
        Full path of the directory where the raw data is stored
        ex/ '/Volumes/QubicData/Calibration/2022-07-14/
    remove_t0 : bool
        subtracts the time of the first sample to the time vector, by default True.

    Returns
    -------
    tt : time for TOD
    tod : the TODs for all detectors
    thk : time for housekeeping data
    az : azimuth of the mount
    el : elevation of the mount
    """
    
    a = qubicfp()
    a.read_qubicstudio_dataset(datadir)
    tt, alltod = a.tod()
    az = a.azimuth()
    el = a.elevation()
    thk = a.timeaxis(datatype='hk')
    if remove_t0:
        ### We remove tt[0]
        tinit = tt[0]
        tt -= tinit
        thk -= tinit
    del(a)
    return tt, alltod, thk, az, el, tinit

def get_azel_moon(ObsSite, tt, tinit, doplot=True):
    MySite = EarthLocation(lat=ObsSite['lat'], lon=ObsSite['lon'], height=ObsSite['height'])
    utcoffset =ObsSite['UTC_Offset']

    dt0 = datetime.utcfromtimestamp(int((tt+tinit)[0]))
    print(dt0)

    nbtime = 100
    tt_hours_loc = tt/3600
    delta_time = np.linspace(np.min(tt_hours_loc), np.max(tt_hours_loc), nbtime)*u.hour

    alltimes = Time(dt0) + delta_time

    ### Local coordinates
    frame_Site = AltAz(obstime=alltimes, location=MySite)

    ### Source
    moon_Site = get_moon(alltimes)
    moonaltazs_Site = moon_Site.transform_to(frame_Site)  

    myazmoon = moonaltazs_Site.az.value
    myelmoon = moonaltazs_Site.alt.value

    azmoon = np.interp(tt_hours_loc, delta_time/u.hour, myazmoon)
    elmoon = np.interp(tt_hours_loc, delta_time/u.hour, myelmoon)
    if doplot:
        plt.figure()
        plt.plot(myazmoon, myelmoon, 'ro')
        plt.plot(azmoon, elmoon)    
    return azmoon, elmoon


def make_coadded_maps(datadir, ObsSite, allTESNum, data=None, speedmin=0.05, 
                      doplot=True, nside=256, az_qubic=0, parallel=False):
    ### First read the data from disk if needed
    if data is None:
        print('Reading data from disk: '+datadir)
        tt, alltod, thk, az, el, tinit = read_data(datadir, remove_t0=True)
        az += az_qubic
        data = [tt, alltod, thk, az, el, tinit]
    else:
        print('Using data already stored in memory - not read from disk')
        tt, alltod, thk, az, el, tinit = data

    ### Azimuth and Elevation of the Moon at the same timestamps from the observing site
    azmoon, elmoon = get_azel_moon(ObsSite, tt, tinit, doplot=doplot)
    
    ### Identify scan types and numbers
    scantype_hk, azt, elt, scantype, vmean = identify_scans(thk, az, el, 
                                                                tt=tt, doplot=doplot, 
                                                                plotrange=[0,2000], 
                                                                thr_speedmin=speedmin)

    ### Loop over TES to do the maps
    print('\nLooping coaddition mapmaking over selected TES')
    print('nside = ',nside)
    start_time = time.perf_counter()
    if parallel is False:
        print('Using sequential loop')
        allmaps = np.zeros((len(allTESNum), 12*nside**2))
        for i in range(len(allTESNum)):
            TESNum = allTESNum[i]
            print('TES# {}'.format(TESNum), end=" ")
            tod = alltod[TESNum-1,:]
            allmaps[i,:], mapscounts = make_coadded_maps_TES(tt, tod, azt, elt, scantype, azmoon, elmoon,
                                                             nside=nside, 
                                                             doplot=doplot, tesnum=TESNum)
            print('OK', flush=True)
    else:
        print('using a parallel loop : no output will be given while processing... be patient...')
        ### Note that this code has been generated using ChatGPT
        def process_TES(i, TESNum, allmaps, alltod, tt, azt, elt, scantype, azmoon, elmoon, nside, doplot):
            # Create a lock for each process to ensure safe access to shared memory
            lock = Lock()
            
            tod = alltod[TESNum - 1, :]
            map_result, mapscounts = make_coadded_maps_TES(tt, tod, azt, elt, scantype, azmoon, elmoon,
                                                           nside=nside, 
                                                           doplot=doplot, tesnum=TESNum)
        
            # Use lock to ensure safe access to shared memory inside the inner function
            with lock:
                # Directly assign the result to the correct index in allmaps
                # allmaps is a list of numpy arrays, so we can use allmaps[i] directly
                allmaps[i] = map_result
        
        def parallel_coadded_maps(allTESNum, alltod, tt, azt, elt, scantype, azmoon, elmoon, nside, doplot):
            # Use Manager to create a shared list that will be modified by parallel processes
            with Manager() as manager:
                # Create a list of NumPy arrays initialized to zeros
                allmaps = manager.list([np.zeros(12 * nside ** 2) for _ in range(len(allTESNum))])
        
                # Run the parallel processing with the correct arguments
                Parallel(n_jobs=-1)(delayed(process_TES)(i, allTESNum[i], allmaps, alltod, tt, azt, elt, scantype, azmoon, elmoon, nside, doplot)
                                    for i in range(len(allTESNum)))
        
                # Convert the manager list back to a NumPy array (this ensures allmaps is a numpy array of arrays)
                allmaps_np = np.array([np.array(allmaps[i]) for i in range(len(allTESNum))])
        
            return allmaps_np

        allmaps = parallel_coadded_maps(allTESNum, alltod, tt, azt, elt, 
                                        scantype, azmoon, elmoon, nside, doplot)
    
    end_time = time.perf_counter()

    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.4f} seconds => average of {(elapsed_time/len(allTESNum)):.4f} per TES")    
        

    # Get central Az and El from pointing
    newazt = (azt - azmoon) * np.cos(np.radians(elt))
    newelt = -(elt - elmoon)
    center=[np.mean(newazt), np.mean(newelt)]
    return allmaps, data, center

def make_coadded_maps_TES(tt, tod, azt, elt, scantype, azmoon, elmoon, nside=256, doplot=True, tesnum=None):
    ###### Pipeline:
    # 1. Remove a slow spline to the data
    tod = tdt.remove_drifts_spline(tt, tod, nsplines=20, doplot=doplot)

    # 2. Offset removal scan by scan using median 
    #    (here we just want to have all scans at the same level before 
    #    decorrelating from azimuth)
    mytod = -tod.copy()
    mytod = remove_offset_scan(mytod, scantype, method='median')

    # 3. Remove azimuth correlation
    # 3.c : splines
    mytod = decorel_azel(mytod, azt, elt, scantype, doplot=doplot, nbins=50, n_el=50, nbspl=5)

    # 4. remove offsets again but this time with mode method as it appears to be less affected 
    #    by the presence of the peaks (no underestimation of the offset resultingg is shadow around the peaks)
    mytod = remove_offset_scan(mytod, scantype, method='mode')

    # Map-making
    newazt = (azt - azmoon) * np.cos(np.radians(elt))
    newelt = -(elt - elmoon)

    # Calculate center of maps from pointing w.r.t. Moon
    center=[np.mean(newazt), np.mean(newelt)]
    
    mapsb, mapcount = healpix_map(newazt[scantype != 0], newelt[scantype != 0], mytod[scantype != 0], nside=nside)
    
    return mapsb, mapcount



In [ ]:
reload(tdt)
nside = 340
azqubic = 116.4

### Check if the data has already been read from disk or not
try:
    data
except NameError:
    data = None
    

allTESNum = [152]
allmaps, data, center = make_coadded_maps(datadir, Obs_Site, allTESNum, data=data, 
                                       nside=nside, doplot=True, az_qubic=azqubic, parallel=False)


In [ ]:
plt.rc('figure',figsize=(20,20))
plt.rc('font',size=12)
n0 = 2
for i in range(len(allTESNum)):
    ok = allmaps[i,:] != hp.UNSEEN
    if (i%(n0**2))==0: 
        plt.tight_layout()
        plt.show()
        plt.figure()
    mm = np.zeros(12*nside**2)+ hp.UNSEEN
    mm[ok] = allmaps[i,ok]-tdt.get_mode(allmaps[i,ok])
    mm[~ok] = hp.UNSEEN
    hp.gnomview(mm, reso=10, sub=(n0,n0,(i%(n0**2))+1), min=-5e3, max=1.2e4, 
                title=allTESNum[i], rot=center)
plt.show()

In [ ]:
reload(tdt)
nside = 340
azqubic = 116.4

### Check if the data has already been read from disk or not
try:
    data
except NameError:
    data = None
    

# allTESNum = [22, 26, 27, 28, 49, 50, 51, 52]
allTESNum = np.arange(256)+1
allmaps, data, center = make_coadded_maps(datadir, Obs_Site, allTESNum, data=data, 
                                       nside=nside, doplot=False, az_qubic=azqubic, parallel=True)


In [ ]:

plt.rc('figure',figsize=(20,20))
plt.rc('font',size=12)
n0 = 8
for i in range(len(allTESNum)):
    ok = allmaps[i,:] != hp.UNSEEN
    if (i%(n0**2))==0: 
        plt.tight_layout()
        plt.show()
        plt.figure()
    mm = np.zeros(12*nside**2)+ hp.UNSEEN
    mm[ok] = allmaps[i,ok]-tdt.get_mode(allmaps[i,ok])
    mm[~ok] = hp.UNSEEN
    hp.gnomview(mm, reso=10, sub=(n0,n0,(i%(n0**2))+1), min=-5e3, max=2.5e4, 
                title=allTESNum[i], rot=center)
plt.show()

In [ ]:
display_healpix_map(allmaps, center, q, reso=10, min=-5e3, max=2.5e4, savepdf='allmoons_2024.pdf')


### We save the maps into a pickle file for subesquent usage.

In [ ]:
import pickle
pickle.dump( [allTESNum, allmaps], open( "NEW2024-allmaps-July14-2022.pkl", "wb" ) )

## Now apply Az/El calibration (from Moon-FWHM-Offsets.ipynb)
The point here is to have each TES directly pointing in the right direction.

What we have done here:

add azqubic=116.4 to the raw file azimuth
work in Moon coordinates:
    newazt = (azt - azmoon) * np.cos(np.radians(elt))
    newelt = -(elt - elmoon)

In [ ]:
def rotate_translate_scale_2d(xin, theta, center, scale):
    rotmat = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    return scale * np.dot(rotmat, (xin-center).T).T

def rot_trans_scale_pts(x, pars):
    pts = np.reshape(x, (len(x)//2, 2))
    return np.ravel(rotate_translate_scale_2d(pts, np.radians(pars[0]), np.array([pars[1],pars[2]]), pars[3]))

def make_coadded_maps_TES(tt, tod, azt, elt, scantype, azmoon, elmoon, nside=256, doplot=True, tesnum=None):
    ###### Pipeline:
    # 1. Remove a slow spline to the data
    tod = tdt.remove_drifts_spline(tt, tod, nsplines=20, doplot=doplot)

    # 2. Offset removal scan by scan using median 
    #    (here we just want to have all scans at the same level before 
    #    decorrelating from azimuth)
    mytod = -tod.copy()
    mytod = remove_offset_scan(mytod, scantype, method='median')

    # 3. Remove azimuth correlation
    # 3.c : splines
    mytod = decorel_azel(mytod, azt, elt, scantype, doplot=doplot, nbins=50, n_el=50, nbspl=5)

    # 4. remove offsets again but this time with mode method as it appears to be less affected 
    #    by the presence of the peaks (no underestimation of the offset resultingg is shadow around the peaks)
    mytod = remove_offset_scan(mytod, scantype, method='mode')

    # Map-making
    newazt = (azt - azmoon) * np.cos(np.radians(elt))
    newelt = -(elt - elmoon)


    ### The file with offsets from Créidhe
    offsets = pickle.load( open( 'pointing_offsets_fixed_hole.pickle', 'rb') )
    offsets_corr = offsets[:,[1,0]]

    #### Fit from Moon-FWHM-Offsets
    fitted_values = np.array([5.048008260655892, -4.551730470265961, 0.6611216259908902, 1.003122495004602])
    ### there is an apparent inversion in this file that is corrected below
    new_offsets = np.reshape(rot_trans_scale_pts(np.ravel(offsets_corr), fitted_values), (256, 2))

    
    # plt.rc('figure',figsize=(16,8))
    # plt.rc('font',size=12)
    # plt.subplot(1,2,1).set_aspect(1)
    # plt.plot(offsets_corr[:,0], offsets_corr[:,1],'bo', alpha=0.2, label='Offsets from Créidhe')
    # plt.plot(new_offsets[:,0], new_offsets[:,1],'ro', alpha=0.2, label='Offest with Moon-fitted correction')
    # plt.legend()
    # plt.xlabel('$\Delta_{az}$ [deg.]')
    # plt.ylabel('$\Delta_{el}$ [deg.]')



    
    mytesn = tesnum
    if np.isfinite(np.sum(new_offsets[mytesn-1,:])):
        newazt = newazt + new_offsets[mytesn-1,0]
        newelt = newelt - new_offsets[mytesn-1,1]   #### The minus sign is not understood !

    # Calculate center of maps from pointing w.r.t. Moon
    center=[np.mean(newazt), np.mean(newelt)]
    
    mapsb, mapcount = healpix_map(newazt[scantype != 0], newelt[scantype != 0], mytod[scantype != 0], nside=nside)
    
    return mapsb, mapcount


In [ ]:
allTESNum = np.arange(256)+1
# allTESNum = [96, 136, 80]
# allTESNum = [22, 26, 27, 28, 49]
allmaps, data, center = make_coadded_maps(datadir, Obs_Site, allTESNum, data=data, 
                                       nside=nside, doplot=False, az_qubic=azqubic, parallel=True)

In [ ]:
plt.rc('figure',figsize=(20,20))
plt.rc('font',size=12)
n0 = 8
for i in range(len(allTESNum)):
    ok = allmaps[i,:] != hp.UNSEEN
    if (i%(n0**2))==0: 
        plt.tight_layout()
        plt.show()
        plt.figure()
    mm = np.zeros(12*nside**2)+ hp.UNSEEN
    mm[ok] = allmaps[i,ok]-tdt.get_mode(allmaps[i,ok])
    mm[~ok] = hp.UNSEEN
    hp.gnomview(mm, reso=3, sub=(n0,n0,(i%(n0**2))+1), min=-5e3, max=2.5e4, 
                title=allTESNum[i])#, rot=center)
plt.show()

In [ ]:
good = np.array([94, 95, 96])-1
display_healpix_map(allmaps, [0,0], q, reso=3, min=-5e3, max=2.5e4, good=good, savepdf='essai.pdf')


In [ ]:
### These are the Offsets from Créidhe tht were used for calibrating the focal plane orientation with the Moon (see above)
offsets = pickle.load( open( 'pointing_offsets_fixed_hole.pickle', 'rb') )
offsets_corr = offsets[:,[1,0]] ### apparent inversion found in Notebook Moon-FWHM-Offsets-2024

#### Fit from Moon-FWHM-Offsets
fitted_values = np.array([5.048008260655892, -4.551730470265961, 0.6611216259908902, 1.003122495004602])
### there is an apparent inversion in this file that is corrected below
new_offsets = np.reshape(rot_trans_scale_pts(np.ravel(offsets_corr), fitted_values), (256, 2))


### This si in xy coordinates with x=Δaz and y=Δel
p = plt.plot(offsets_corr[:,0], offsets_corr[:,1], 'o', label='Maynooth File')
p = plt.plot(new_offsets[:,0], new_offsets[:,1], 'o', label='Maynooth File Moon rotated')
plt.legend()

### For a nice plot we need to go to polar coordinates where theta will be the radial distance
rth = np.sqrt(offsets_corr[:,0]**2 + offsets_corr[:,1]**2)
ph = np.arctan2(offsets_corr[:,1], offsets_corr[:,0])
newrth = np.sqrt(new_offsets[:,0]**2 + new_offsets[:,1]**2)
newph = np.arctan2(new_offsets[:,1], new_offsets[:,0])


fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
plt.plot(ph, rth, 'o', label='Maynooth file')
plt.plot(newph, newrth, 'o', label='Maynooth file Moon rotated')
plt.legend()

In [ ]:
#### These are files found in the calfiles directry of QubicSoft.
#### They should contain the synthesized beam related informations:
#### - peak positions and amplitude as a function of frequency for all detectors and for 150 and 220 GHz
#### => they apparently contain mosly crap... this is weird and needs to be fixed.

plt.rc('figure',figsize=(10,6))
plt.rc('font',size=12)

from qubic.lib.Calibration.Qcalibration import QubicCalibration
newd = d.copy()

newd['synthbeam'] = 'CalQubic_Synthbeam_Analytical_220_FI.fits'              ## (992, 9)
# newd['synthbeam'] = 'CalQubic_Synthbeam_Analytical_Multifreq_MJW_FI.fits'       ## (11, 992, 9)
# newd['synthbeam'] = 'CalQubic_Synthbeam_Calibrated_JCH_FI.fits'                 ## (10, 9)
# newd['synthbeam'] = 'CalQubic_Synthbeam_Calibrated_Multifreq_FI.fits'           ## (15, 10, 9)
# newd['synthbeam'] = 'CalQubic_Synthbeam_Maynooth_220_FI.fits'                   ## (992, 9)


print(newd['synthbeam'])
c = QubicCalibration(newd)
thetafits,phifits,valfits,freqs, mheader = c.get('synthbeam')
print('\n Thetafits:', thetafits.shape)
print('\n Phifits:', phifits.shape)
print('\n Valfits:', valfits.shape)
print('\n freqs', np.shape(freqs), '\n', freqs)
print('\n mheader\n', mheader)
print()
print(c.nu)


fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
for idet in range(256):
    col = None
    for ipk in range(4,5):
        if (idet+ipk) == 0:
            lab = newd['synthbeam']
        else:
            lab = None
        # p = plt.plot(phifits[:, idet, ipk], np.degrees(thetafits[:, idet, ipk]), 'o', color=col)
        p = plt.plot(phifits[idet, ipk], np.degrees(thetafits[idet, ipk]), 'o', color=col, label=lab)
        # highpeak = np.argmax(valfits[idet,:])
        # p = plt.plot(phifits[idet, highpeak], np.degrees(thetafits[idet, highpeak]), 'o', color=col)
        col = p[0].get_color()
plt.grid(True)
plt.plot(ph, rth, 'x', label='Maynooth file')
plt.plot(newph, newrth, 'x', label='Maynooth file Moon Rotated')
plt.legend()

In [ ]:
# It does not look totally absurd, but I guess in the calfiles, we're not taking the L.O.S. detectors but the first one in the list... 
# Also there seem to be rotation...

In [ ]:
### now let's calculate L.O.S. for all detectors from their focal plane position (this is what is used in principle by default in the software if no calfile is used)

import numexpr as ne
position = q.detector.center
myposition =  -position / np.sqrt(np.sum(position**2, axis=-1))[..., None]
local_dict = {'nx': myposition[:, 0, None], 'ny': myposition[:, 1, None]}
thlos = ne.evaluate('arcsin(sqrt(nx**2 + ny**2))',
            local_dict=local_dict)
phlos = ne.evaluate('arctan2(ny, nx)', local_dict=local_dict)

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
for idet in range(256):
    col = None
    for ipk in range(1):
        if (idet+ipk) == 0:
            lab = newd['synthbeam']
        else:
            lab = None
        # p = plt.plot(phifits[:, idet, ipk], np.degrees(thetafits[:, idet, ipk]), 'o', color=col)
        p = plt.plot(phifits[idet, ipk], np.degrees(thetafits[idet, ipk]), 'o', color=col, label=lab)
        # highpeak = np.argmax(valfits[idet,:])
        # p = plt.plot(phifits[idet, highpeak], np.degrees(thetafits[idet, highpeak]), 'o', color=col)
        col = p[0].get_color()
plt.grid(True)
plt.plot(ph, rth, 'x', color='r', label='Maynooth file')
plt.plot(newph, newrth, 'x', label='Maynooth file Moon Rotated')
plt.plot(phlos, np.degrees(thlos), '+', color='b', label='QubicSoft')
plt.legend()


In [ ]:
### So it's a real mess... we have to figure that out !